# Aggregation and Grouping

An essential peice of analysis of large data is efficient summarization: computing aggregations like `sum()`, `mean()`, `median()`, `min()` and `max()`, in which a single number gives insight into the nature of a potentially large dataset. In this section, we'll explore aggreagtions in Pandas, from simple operations akin to what we've seen on NumPy arrays, to more sophisticated operations based on the concept of a `groupby`.

For convenience, we'll use the same `display`magic function that we've seen in previous sections:

In [53]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)



## Planets Data

Here we wil luse the Planets dataset, available via the seaborn package. It gives information on planets that astronomers have discovered around other stars (known *as extrasolar planets* or *exoplanets* for short).

In [3]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [4]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


This has some details on the $1000+$ extrasolar planets discovered up to 2014.

## Simple Aggregation in Pandas

Earlier, we explored some of the data aggregations available for NumPy arrays. As with a one-dimensional NumPy array, for a Pandas `Series` the aggregates return a single value:

In [5]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [6]:
ser.sum()

2.811925491708157

In [ ]:
ser.mean()

For a `DataFrame,` by default the aggregates return results within each column:

In [8]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df                   

,A,B
0,0.183405,0.611853
1,0.304242,0.139494
2,0.524756,0.292145
3,0.431945,0.366362
4,0.291229,0.456070


In [9]:
df.mean()

A    0.347115
B    0.373185
dtype: float64

In [10]:
df.mean(axis=1)

0    0.397629
1    0.221868
2    0.408451
3    0.399153
4    0.373650
dtype: float64

Pandas `Series` and `DataFrame`s include all of the common aggregates mentioned earlier; in addition, there is a convenience method `describe()` that computes several common aggregates for each column and returns the result. Let's use this on the Planets data, for now dropping rows with missing values:

In [11]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


This can be a useful way to begin understanding the overall properties of a dataset. For example, we see in the `year` column that although exoplanets were discovered as far back as 1989, half fo all known exoplanets were not discovered until 2010 or after. This is largely thanks to the *Kepler* mission, which is a space-based telescope specifically designed for finding eclipsing planets around other stars.

The following table summarizes other built-in Pandas aggregations:

```
Aggregation 	Description
count() 	Total number of items
first(), last() 	First and last item
mean(), median() 	Mean and median
min(), max() 	Minimum and maximum
std(), var() 	Standard deviation and variance
mad() 	Mean absolute deviation
prod() 	Product of all items
sum() 	Sum of all items

```

These are all methods of `DataFrame` and `Series` objects.

To go deeper into the data, however, simple aggregates are often not enough. The next level of data summarization is the `groupby` operation, which allows you to quickly and efficiently compute aggregates on subsets of data.

## GroupBy: Split, Apply, Combine

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called `groupby` operation. The name "groub by" comes from a command in the SQL database language, but it is perhaps more illuminative to think of it in the terms first coniend by Hadley Wickham of Rstats fame: `split, apply, combine`.

### Split, apply, combine

This makes clear what `groupby` accomplished:

- The *split* step involves breaking up and grouping a `DataFrame` depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, whithin the individual groups.
- The *combine* step merges the results of these operations into an output array.

While this could certainly be done manually using some combination of the masking, aggregation, and mergin commands covered earlier, an important realization is that the *intermediate splits do not need to be explicitly intantiated*. Rather, the `GroupBy` can (often) do this in a single pass over the data, updating the sum, mean, count, min, or other aggregate for each group along the way. The power of the `GroupBy` is that is abstracts away these steps: the user need not think about *how* the computation is done under the hood, but rahter thinks about the *operation as a whole*.

As a concrete example, let's take a look at using Pandas for the computation shown in this diagram. We'll star tby creating the input `DataFrame`:

In [20]:
df = pd.DataFrame({'key': list('ABCABC'),
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


The most basic split-apply-combine operation can be computed with the `groupby()` method of `DataFrame`s, passing the name of the desired key column:

In [30]:
df.groupby('key')

The `sum()` method is just one possibility here; you can apply virtually any common Pandas or NumPy aggregation function, as well as virtually any valid `DataFrame` operation, as we will see in the following discussion.

### The GroupBy object

The `GroupBy` object is a very flexible abstraction. In many ways, you can simply treat it as if it's a collection of `DataFrame`s, and it does the difficult things under the hood. Let's see some examples using the Planets data.

### Column indexing

The `GroupBy` object supports column indexing in the same way as the `DataFrame`, and returns a modified `GroupBy` object.

In [32]:
planets.groupby('method')

In [34]:
planets.groupby('method')['orbital_period']

Here we've selected a particular `Series` group from the original `DataFrame` group by reference to its column name. As with the `GroupBy` object, no computation is done until we call some aggregate on the object:

In [35]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

That gives an idea of the general scale of orbital periods (in days) that each method is sensitive to.

#### Iteration over groups

The `GroupBy` object supports direct iteration over the groups, returning each group as a `Series` or `DataFrame`:

In [38]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


This can be useful for doing certain things manually, though it is often much faster to sue the built-in `apply` functionality, which we will discuss momentarily.

#### Dispatch methods

Thorugh some Python class magic, any method not explicitly implemented by the `GroupBy` object will be passed through and called on the groups, whether they are `DataFrame` or `Series` objects. For example, you can sue the `describe()` method of `DataFrame`s to perform a set of aggreagtions that describe each grop in the data:

In [86]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.0
       Eclipse Timing Variations           9.0
       Imaging                            38.0
       Microlensing                       23.0
       Orbital Brightness Modulation       3.0
                                         ...  
max    Pulsar Timing                    2011.0
       Pulsation Timing Variations      2007.0
       Radial Velocity                  2014.0
       Transit                          2014.0
       Transit Timing Variations        2014.0
Length: 80, dtype: float64

Looking at this table help us to better understand the data: for example, the vast majority of planets have been discovered by the Radial Velocity and transit methods, though the latter only became common (due to new, more accurate telescopes) in the last decade. The newest methods seem to be Transit Timing Variation on and Orbital Brightness Modulation, which were not used to discover a new planet until 2011.

This is just one example of the utiliy of dispatch methods. Notice that they are applied to *each individual group*, and the results are the combined within `GroupBy` and returned. Again, any valid `DataFrame / Series` method can be used on the corresponding `GroupBy` object, which allows for some very flexible and powerful operations.

### Aggregate, filter, transform, apply

The preceding discussion focused on aggregation of the combine operation, but there are more options available. In particular, `GroupBy` objects have `aggregate()`, `filter()`, `transform()`, and `apply()` methods that efficiently implementa a variety of useful operations before combining the grouped data.

For the purpose of the following subsections, we'll use this `DataFrame`:

In [45]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': list('ABCABC'),
                   'data1': range(6),
                   'data2': rng.randint(0, 16, 6)},
                   columns= ['key', 'data1', 'data2'])
df                   

,key,data1,data2
0,A,0,12
1,B,1,15
2,C,2,5
3,A,3,0
4,B,4,3
5,C,5,11


#### Aggregation

We're now familiar with `GroupBy` aggregations with `sum()`, `median()`, and the like, but the `aggregate()` method allows for even more flexibility. It can take a string, a function, or alist thereof, and compute all the aggregates at once. Here is a quick example combining all these:

In [46]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     0      6  12
B       1    2.5   4     3      9  15
C       2    3.5   5     5      8  11

Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column:

In [49]:
df.groupby('key').aggregate({'data1': 'min', 'data2': 'max'})
                              

,data1,data2
key,,
A,0,12
B,1,15
C,2,11


#### Filtering

A filtering operation allows you to drop data based on the group properties. For example , we might want to keep all gorups in which the standard deviation is larger than some critical value:

In [56]:
def filter_func(x):
    return x['data2'].std() > 5

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0     12
1   B      1     15
2   C      2      5
3   A      3      0
4   B      4      3
5   C      5     11

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  8.485281
B    2.12132  8.485281
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
0   A      0     12
1   B      1     15
3   A      3      0
4   B      4      3

The filter function should return a Boolean value specifying whether the gropu passed the filtering. There because group A does not have a standard deviation greater than 4, it is dropped from the result.

#### Transformation

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine. For such a transformation, the output is the same shape as the input. A common example is to center the data by subtracting the group-wise mean:

In [57]:
df.groupby('key').transform(lambda x: x- x.mean())

,data1,data2
0,-1.5,6.0
1,-1.5,6.0
2,-1.5,-3.0
3,1.5,-6.0
4,1.5,-6.0
5,1.5,3.0


#### The apply() method

The `apply()` method lets you apply an arbitrary function to the group results. The function should take a `DataFrame`, and return either a Pandas object (e.g, `DataFrame`, `Series`) or a scalar; the combine opration will be tailored to the type of output returned.

For example, here is an `apply()` that normalizes the first column by the sum of the second:

In [59]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,12
1,B,1,15
2,C,2,5
3,A,3,0
4,B,4,3
5,C,5,11
,key,data1,data2
0,A,0.000000,12
1,B,0.055556,15
2,C,0.125000,5


`apply()` within a `Groupby` is quite flexible: the only criterion is that the function takes a `DataFrame` and returns a Pandas object or scalar; what you do in the middle is up to you.

### Specifying the split key

In the simple examples presented before, we split the `DataFrame` on a single column name. This is just one of many options by which the groups can be defined, and we'll go through some other options for group specification here.

#### A list, array, series, or index providing the grouping keys

They key can be any series or list with a length matching that of the `DataFrame`. For example:

In [65]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0     12
1   B      1     15
2   C      2      5
3   A      3      0
4   B      4      3
5   C      5     11

df.groupby(L).sum()
   data1  data2
0      7     28
1      4     15
2      4      3

Of course, this means there's another, more verbose way of accomplishing the `df.grouby('key')` from before:

In [66]:
display('df', "df.groupby(df['key']).sum()")

df
  key  data1  data2
0   A      0     12
1   B      1     15
2   C      2      5
3   A      3      0
4   B      4      3
5   C      5     11

df.groupby(df['key']).sum()
     data1  data2
key              
A        3     12
B        5     18
C        7     16

#### A dictionary or series mapping index to group

Another method is to provide a dictionary that maps index values to the group keys:

In [67]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,12
B,1,15
C,2,5
A,3,0
B,4,3
C,5,11
,data1,data2
consonant,12,34
vowel,3,12


#### Any Python function

Similar to mapping, you can pass any Python function that will input the index value and output the group:

In [69]:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2
key,,
A,0,12
B,1,15
C,2,5
A,3,0
B,4,3
C,5,11
,data1,data2
a,1.5,6
b,2.5,9


#### A list of valid keys

Further, any of the preceding key choices can be combined to group on a multi-index:

In [71]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,6
b,consonant,2.5,9
c,consonant,3.5,8


### Grouping example

As an example of this, in a couple lines of Python code we can put all these togehter and count discovered planets by method and by decade:

In [87]:
decade = 10 * (planets['year'] // 10)

decade = decade.astype(str) + 's'
decade.name = 'decade'
decade

planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


This shows the power of combining many of the operations we've discussed up to this point when looking at realistic datasets. We immediately gain a coarse understanding of then and how planets have been discovered over the past several decades!

Here I would suggest digging into these few lines of code, and evaluating the individual steps to make sure you understand exactly what they are doing to the result. It's certainly a somewhat complicated example, but understanding these pices will give you the means to similarly explore your own data.